## Linking a dataset of real historical persons 

In this example, we deduplicate a more realistic dataset.  The data is based on historical persons scraped from wikidata.  Duplicate records are introduced with a variety of errors introduced.

In [1]:
from splink.datasets import splink_datasets
import altair as alt
import pandas as pd

df = splink_datasets.historical_50k
df.head()


,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [2]:
from splink.database_api import DuckDBAPI
from splink.profile_data import profile_columns
from splink.column_expression import ColumnExpression

db_api = DuckDBAPI()
profile_columns(df, db_api, column_expressions=["first_name", "substr(surname,1,2)"])


alt.VConcatChart(...)

In [3]:
from splink.blocking_rule_library import block_on
from splink.linker import Linker
# Simple settings dictionary will be used for exploratory analysis
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name"),
    ],
}

linker = Linker(df, settings, database_api=db_api)
linker.cumulative_num_comparisons_from_blocking_rules_chart()



alt.Chart(...)

In [4]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name"),
    ],
    "comparisons": [
        ctl.NameComparison("first_name").configure(term_frequency_adjustments=True),
        ctl.NameComparison("surname").configure(term_frequency_adjustments=True),
        # TODO: Restore ctl.DateComparison level here
        cl.LevenshteinAtThresholds("dob"),
        # TODO: Restore ctl.PostcodeComparison level here
        cl.LevenshteinAtThresholds("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = Linker(df, settings, database_api=db_api)

In [5]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.000136.
This means that amongst all possible pairwise record comparisons, one in 7,362.31 are expected to match.  With 1,279,041,753 total possible comparisons, we expect a total of around 173,728.33 matching pairs


In [6]:
linker.estimate_u_using_random_sampling(max_pairs=5e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


In [7]:
training_blocking_rule = block_on("first_name", "surname")
training_session_names = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Iteration 1: Largest change in params was -0.53 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.0347 in the m_probability of birth_place, level `All other comparisons`
Iteration 3: Largest change in params was 0.012 in the m_probability of birth_place, level `Exact match on birth_place`
Iteration 4: Largest change in params was 0.00376 in the m_probability of birth_place, level `Exact match on birth_place`

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - first_na

In [8]:
training_blocking_rule = block_on("dob")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.362 in the m_probability of first_name, level `Exact match on first_name`
Iteration 2: Largest change in params was 0.0335 in the m_probability of first_name, level `All other comparisons`
Iteration 3: Largest change in params was 0.00478 in the m_probability of first_name, level `All other comparisons`

EM converged after 3 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


The final match weights can be viewed in the match weights chart:

In [9]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [10]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [11]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key
0,-19.318452,0.000002,Q7528834-11,Q969969-1,sir,sir,3,0.024985,0.024985,42.297448,...,0.155044,1.0,politician,businessperson,0,0.088932,0.015738,0.105709,1.0,0
1,-19.318452,0.000002,Q7529472-2,Q969969-1,sir,sir,3,0.024985,0.024985,42.297448,...,0.155044,1.0,surgeon,businessperson,0,0.007790,0.015738,0.105709,1.0,0
2,-19.318452,0.000002,Q7528038-2,Q969969-1,sir,sir,3,0.024985,0.024985,42.297448,...,0.155044,1.0,politician,businessperson,0,0.088932,0.015738,0.105709,1.0,0
3,-19.318452,0.000002,Q61641393-6,Q969969-1,sir,sir,3,0.024985,0.024985,42.297448,...,0.155044,1.0,barrister,businessperson,0,0.005734,0.015738,0.105709,1.0,0
4,-19.318452,0.000002,Q61641393-3,Q969969-1,sir,sir,3,0.024985,0.024985,42.297448,...,0.155044,1.0,writer,businessperson,0,0.053264,0.015738,0.105709,1.0,0


You can also view rows in this dataset as a waterfall chart as follows:

In [12]:
from splink.charts import waterfall_chart
records_to_plot = df_e.to_dict(orient="records")
linker.waterfall_chart(records_to_plot, filter_nulls=False)

alt.LayerChart(...)

In [13]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.95)

Completed iteration 1, root rows count 628
Completed iteration 2, root rows count 106
Completed iteration 3, root rows count 24
Completed iteration 4, root rows count 6
Completed iteration 5, root rows count 0


In [14]:
linker.cluster_studio_dashboard(df_predict, clusters, "dashboards/50k_cluster.html", sampling_method='by_cluster_size', overwrite=True)

from IPython.display import IFrame

IFrame(
    src="./dashboards/50k_cluster.html", width="100%", height=1200
)

In [18]:
linker.roc_chart_from_labels_column("cluster",match_weight_round_to_nearest=0.02)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'surname':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


alt.Chart(...)

In [19]:
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.999,
    include_false_negatives=False,
    include_false_positives=True,
).as_record_dict()
linker.waterfall_chart(records)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'surname':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


alt.LayerChart(...)

In [20]:
# Some of the false negatives will be because they weren't detected by the blocking rules
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.5,
    include_false_negatives=True,
    include_false_positives=False,
).as_record_dict(limit=50)

linker.waterfall_chart(records)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'surname':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


alt.LayerChart(...)